# Asynchronous Programming in Scala

## Key Concepts

*   **Processes, Threads, Tasks**: Understanding the layers of execution.
    *   A **Process** is an instance of a program running, with its own memory space.
    *   A **Thread** is the smallest unit of execution within a process, with its own stack but sharing the process's memory.
    *   A **Task** (in the context of async runtimes or thread pools) is a unit of work scheduled to run on a thread.
*   **Synchronous vs. Asynchronous**: Blocking vs. non-blocking execution.
*   **Blocking**: An operation that halts the current thread until it completes.
*   **Non-Blocking**: An operation that allows the current thread to continue while the operation runs elsewhere (another thread or I/O event loop).
*   **Futures**: A placeholder object for a result that is not yet available, representing a computation that may complete in the future.
*   **ExecutionContext**: Scala's abstraction over thread pools, responsible for executing `Future` computations.
*   **Eager vs Lazy Execution**: A key difference between Scala Futures and Rust Futures.

![Components](thread-components.png)

![Hierarchy](thread-task-hierarchy.png)

Hierarchy: see file async_hierarchy_panics.scala


In [1]:
import sys.process._
import scala.concurrent._
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global
import java.time.{Duration => JDuration, Instant}
import java.util.concurrent.{Executors, CopyOnWriteArrayList}
import scala.util.{Try, Success, Failure}

import sys.process._

import scala.concurrent._

import scala.concurrent.duration._

import scala.concurrent.ExecutionContext.Implicits.global

import java.time.{Duration => JDuration, Instant}

import java.util.concurrent.{Executors, CopyOnWriteArrayList}

import scala.util.{Try, Success, Failure}


## Scala Futures

Scala's `Future` represents a computation that will eventually complete with a result (`Success`) or a failure (`Failure`).

Creating a `Future` in Scala is done using the `Future { ... }` block. This requires an implicit `ExecutionContext` to know where to run the code.


In [5]:
println("1. Creating a basic Future:")

val basicFuture: Future[Int] = Future {
  println("  → Computing a result in a Future...")
  Thread.sleep(1500) // Simulate some work
  42 // The result of the Future
}
// Thread.sleep(10)
println("  → Future created. The computation might already be running.")


1. Creating a basic Future:
  → Future created. The computation might already be running.
  → Computing a result in a Future...


basicFuture: Future[Int] = Success(value = 42)

### Getting Results (Synchronously vs. Asynchronously)

There are two main ways to obtain the result of a `Future`:

1.  **Synchronously (Blocking):** Using `Await.result`. This *blocks* the current thread until the Future completes. **Generally, avoid this in production asynchronous code** as it defeats the purpose of async and can lead to deadlocks or performance issues, especially on limited thread pools. It's primarily for testing or bridging async code back to blocking code (like in a `main` method or a legacy sync API).
3.  **Asynchronously (Non-Blocking):** Using callbacks (`onComplete`) or combinators (`map`, `flatMap`, `for-comprehension`). This is the preferred method in asynchronous applications.

Let's demonstrate `Await.result` and then immediately show the non-blocking alternatives.

In [14]:
// 1. AWAITING RESULTS (DEMONSTRATION ONLY - AVOID IN PROD)
println("\n1. Awaiting results (avoid in production code):")
try {
  // Await.result blocks the current thread!
  val result = Await.result(basicFuture, 2.seconds)
  println(s"  → Await result: $result")
} catch {
  case e: TimeoutException => println("  → Future timed out!")
  case e: Exception => println(s"  → Future failed: ${e.getMessage}")
}

// 2. CALLBACKS - Non-blocking way to handle results
println("\n2. Using callbacks (non-blocking):")

val anotherFuture: Future[String] = Future {
  println("  → Another Future computing...")
  Thread.sleep(600)
  "Hello from the Future!"
}

// In Scala 3 (and preferred in Scala 2), use onComplete to handle both Success and Failure
anotherFuture.onComplete { // Called when the Future completes, regardless of success or failure
  case Success(value) => println(s"  → onComplete Success: '$value'")
  case Failure(exception) => println(s"  → onComplete Failure: '${exception.getMessage}'")
} // No need to pass ExecutionContext.Implicits.global explicitly if it's already in scope

// Example using onComplete with pattern matching for a Future that might succeed
val yetAnotherFuture: Future[Int] = Future {
  println("  → Yet another Future...")
  Thread.sleep(700)
  100
}

yetAnotherFuture.onComplete {
  case Success(value) => println(s"  → onComplete (pattern match): Result is $value")
  case Failure(exception) => println(s"  → onComplete (pattern match): Failed with ${exception.getMessage}")
}

// Wait a bit to let the non-blocking callbacks execute before the cell finishes
Thread.sleep(1000)


1. Awaiting results (avoid in production code):
  → Await result: 42

2. Using callbacks (non-blocking):
  → Another Future computing...
  → Yet another Future...
  → onComplete Success: 'Hello from the Future!'
  → onComplete (pattern match): Result is 100


anotherFuture: Future[String] = Success(value = "Hello from the Future!")
yetAnotherFuture: Future[Int] = Success(value = 100)

**Key takeaway:** Use non-blocking methods (`onComplete`, `map`, `flatMap`) whenever possible in asynchronous code.

`Await.result` is generally an anti-pattern in such contexts.

In [17]:
// EAGER FUTURES (DEFAULT IN SCALA)
def lionRuns(): Future[Unit] = Future {
  println("🦁 Lion is running!")
}

def foxRuns(): Future[Unit] = Future {
  println("🦊 Fox is running!")
}

def rabbitRuns(): Future[Unit] = Future {
  println("🐇 Rabbit is running!")
}

println("🦁 lionRuns() is called.")
val lionFuture = lionRuns() // Execution begins immediately!
println("🦊 foxRuns() is called.")
val foxFuture = foxRuns()   // Execution begins immediately!
println("🐇 rabbitRuns() is called.")
val rabbitFuture = rabbitRuns() // Execution begins immediately!
println("(All animals are running already in Scala Future!)")

// We only await some of the futures
Await.result(foxFuture, 2.seconds)
Await.result(lionFuture, 2.seconds)

// We never await the rabbit, but it still runs!
println("All animals have finished running (even the rabbit 🐇, because Scala Future is eager)!")

🦁 lionRuns() is called.
🦊 foxRuns() is called.
🦁 Lion is running!
🐇 rabbitRuns() is called.
🦊 Fox is running!
🐇 Rabbit is running!
(All animals are running already in Scala Future!)
All animals have finished running (even the rabbit 🐇, because Scala Future is eager)!


defined function lionRuns
defined function foxRuns
defined function rabbitRuns
lionFuture: Future[Unit] = Success(value = ())
foxFuture: Future[Unit] = Success(value = ())
rabbitFuture: Future[Unit] = Success(value = ())

## 2. Thread Pools (`ExecutionContext`)

Futures run their computations on `ExecutionContexts`, which typically wrap thread pools. The global `ExecutionContext.Implicits.global` uses a `ForkJoinPool`, suitable for CPU-bound tasks, but can be saturated by blocking I/O.

You can create custom thread pools for different types of tasks (e.g., a fixed thread pool for CPU-bound tasks, a cached or larger pool for I/O-bound tasks).

This example (adapted from `async_threadpool.scala`) shows how thread pool size affects the execution time of CPU-bound tasks simulated by `Thread.sleep`.

In [19]:
import scala.concurrent._
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext
import java.util.concurrent.Executors
import java.time.Instant
import java.time.Duration

// Simulate a CPU-bound task by blocking a thread
def cpuBoundTask(name: String): Unit = {
  println(s"  $name starting work...")
  Thread.sleep(1000) // Blocks the thread for 1 second
  println(s"  $name finished!")
}

def runWithThreads(threadCount: Int): Unit = {
  println(s"\n=== Running with $threadCount thread(s) ===\n")
  val start = Instant.now()

  // Create a custom thread pool
  val executorService = Executors.newFixedThreadPool(threadCount)
  implicit val ec: ExecutionContext = ExecutionContext.fromExecutorService(executorService)

  try {
    // Create three futures that will run our CPU-bound task
    // They will run concurrently up to the thread pool size
    val lion = Future { cpuBoundTask("🦁 Lion") }
    val fox = Future { cpuBoundTask("🦊 Fox") }
    val rabbit = Future { cpuBoundTask("🐇 Rabbit") }

    // Wait for all futures to complete
    // Future.sequence combines a Seq[Future[T]] into a Future[Seq[T]]
    val allAnimalsFuture = Future.sequence(Seq(lion, fox, rabbit))

    // Await the combined future for demonstration purposes
    Await.result(allAnimalsFuture, 10.seconds)

    // Calculate duration using java.time.Duration
    val duration = Duration.between(start, Instant.now()).toMillis / 1000.0
    println(s"\nAll animals finished! Total time with $threadCount thread(s): $duration seconds")

  } finally {
    // Always shut down custom thread pools!
    executorService.shutdown()
  }
}

// Run with different thread pool sizes to see the effect
runWithThreads(1) // Tasks run sequentially
Thread.sleep(100)
runWithThreads(3) // Tasks run in parallel (if 3+ cores available)
Thread.sleep(100)


=== Running with 1 thread(s) ===

  🦁 Lion starting work...
  🦁 Lion finished!
  🦊 Fox starting work...
  🦊 Fox finished!
  🐇 Rabbit starting work...
  🐇 Rabbit finished!

All animals finished! Total time with 1 thread(s): 3.014 seconds

=== Running with 3 thread(s) ===

  🦁 Lion starting work...
  🦊 Fox starting work...
  🐇 Rabbit starting work...
  🦁 Lion finished!
  🐇 Rabbit finished!
  🦊 Fox finished!

All animals finished! Total time with 3 thread(s): 1.006 seconds


import scala.concurrent._

import scala.concurrent.duration._

import scala.concurrent.ExecutionContext

import java.util.concurrent.Executors

import java.time.Instant

import java.time.Duration

// Simulate a CPU-bound task by blocking a thread

defined function cpuBoundTask
defined function runWithThreads

**Observation:**
*   With 1 thread, the tasks run one after another, taking roughly `3 * 1 second = 3 seconds`.
*   With 3 threads, the tasks can run in parallel, taking roughly `max(1 second, 1 second, 1 second) = 1 second` (plus overhead).

This demonstrates that the thread pool size directly impacts the parallelism of CPU-bound tasks. For I/O-bound tasks (which spend most time waiting, not computing), fewer threads can be sufficient as threads can switch when waiting.

## Eager vs Lazy Execution

One fundamental difference between Scala's `Future` and Rust's async/await is **execution behavior**:

- **Scala Futures are eager**: They start executing as soon as they're created
- **Rust futures are lazy**: They only start when explicitly awaited

**File: `await.scala`**

### Scala Futures: Key Patterns and Best Practices

In [8]:
import scala.concurrent.Future
import scala.concurrent._
import ExecutionContext.Implicits.global
import scala.util.{Failure, Success}
import scala.concurrent.duration._

// 1. CREATING FUTURES
println("1. Creating a basic Future:")
val basicFuture: Future[Int] = Future {
  println("  → Computing a result...")
  Thread.sleep(1000) // Simulate work
  42 // The result
}

// 2. CALLBACKS - Non-blocking way to handle results
println("\n2. Using callbacks (non-blocking):")
basicFuture.onComplete {
  case Success(value) => println(s"  → Success callback: The result is: $value")
  case Failure(exception) => println(s"  → Failure callback: An error occurred: ${exception.getMessage}")
}

// 3. TRANSFORMATIONS - Chaining operations
println("\n3. Transforming futures:")
val transformedFuture = basicFuture
  .map(result => result * 2)
  .map(doubled => s"The doubled result is: $doubled")

transformedFuture.onComplete {
  case Success(message) => println(s"  → $message")
  case Failure(exception) => println(s"  → Transform failed: ${exception.getMessage}")
}

// 4. COMPOSING FUTURES
println("\n4. Composing multiple futures:")
def getUsername(): Future[String] = Future { "user123" }
def getUserData(username: String): Future[Map[String, String]] = Future { 
  Map("name" -> "John Doe", "username" -> username) 
}

// Using flatMap for sequential composition
val userDataFuture = getUsername().flatMap(username => getUserData(username))

// The same thing using for-comprehension (more readable)
val userDataFuture2 = for {
  username <- getUsername()
  userData <- getUserData(username)
} yield userData

userDataFuture.onComplete {
  case Success(data) => println(s"  → User data: $data")
  case Failure(e) => println(s"  → Failed to get user data: ${e.getMessage}")
}

// 5. ERROR HANDLING
println("\n5. Error handling:")
val failingFuture: Future[String] = Future {
  throw new RuntimeException("Something went wrong!")
  "This will never be returned"
}

failingFuture
  .recover { case ex: RuntimeException => s"Recovered from: ${ex.getMessage}" }
  .onComplete {
    case Success(result) => println(s"  → Recovery result: $result")
    case Failure(ex) => println(s"  → Recovery failed: ${ex.getMessage}")
  }

// 6. AWAITING RESULTS (FOR DEMONSTRATION ONLY)
println("\n6. Awaiting results (avoid in production code):")
try {
  val result = Await.result(basicFuture, 2.seconds)
  println(s"  → Await result: $result")
} catch {
  case e: TimeoutException => println("  → Future timed out!")
  case e: Exception => println(s"  → Future failed: ${e.getMessage}")
}

println("\n=== BEST PRACTICES ===")
println("• Avoid blocking with Await.result in production code")
println("• Use callbacks and transformations (map/flatMap) instead")
println("• Handle errors explicitly with recover or recoverWith")
println("• Use for-comprehensions for multiple sequential futures")
println("• Consider custom ExecutionContexts for production apps")

// Sleep to ensure callbacks have time to execute
Thread.sleep(2000)

1. Creating a basic Future:

2. Using callbacks (non-blocking):
  → Computing a result...

3. Transforming futures:

4. Composing multiple futures:

5. Error handling:
  → User data: Map(name -> John Doe, username -> user123)

6. Awaiting results (avoid in production code):
  → Recovery result: Recovered from: Something went wrong!
  → Await result: 42

=== BEST PRACTICES ===
  → Success callback: The result is: 42
  → The doubled result is: 84
• Avoid blocking with Await.result in production code
• Use callbacks and transformations (map/flatMap) instead
• Handle errors explicitly with recover or recoverWith
• Use for-comprehensions for multiple sequential futures
• Consider custom ExecutionContexts for production apps


import scala.concurrent.Future

import scala.concurrent._

import ExecutionContext.Implicits.global

import scala.util.{Failure, Success}

import scala.concurrent.duration._

// 1. CREATING FUTURES

basicFuture: Future[Int] = Success(value = 42)
transformedFuture: Future[String] = Success(value = "The doubled result is: 84")
defined function getUsername
defined function getUserData
userDataFuture: Future[Map[String, String]] = Success(value = Map("name" -> "John Doe", "username" -> "user123"))
userDataFuture2: Future[Map[String, String]] = Success(value = Map("name" -> "John Doe", "username" -> "user123"))
failingFuture: Future[String] = Failure(exception = java.lang.RuntimeException: Something went wrong!)

## 2. Thread Pool Effects on Performance

Thread pool size affects how many CPU-bound tasks can execute in parallel. This example demonstrates the effect of different thread pool sizes.

**File: `async_threadpool.scala`**

In [4]:
// Define our CPU-bound task simulator
def cpuBoundTask(name: String): Unit = {
  println(s"$name starting work...")
  Thread.sleep(1000) // Simulate CPU-bound work
  println(s"$name finished!")
}

// Demo with 1 thread
def runWithThreads(threadCount: Int): Unit = {
  println(s"\n=== Running with $threadCount thread(s) ===\n")
  val start = Instant.now()
  
  // Create custom thread pool
  val executorService = Executors.newFixedThreadPool(threadCount)
  val ec = ExecutionContext.fromExecutorService(executorService)
  
  try {
    // Create three futures for our animals
    implicit val executionContext = ec
    
    val lion = Future {
      cpuBoundTask("🦁 Lion")
    }
    
    val fox = Future {
      cpuBoundTask("🦊 Fox")
    }
    
    val rabbit = Future {
      cpuBoundTask("🐇 Rabbit")
    }
    
    // Wait for all to complete
    Await.result(Future.sequence(Seq(lion, fox, rabbit)), 10.seconds)
    
    val duration = JDuration.between(start, Instant.now()).toMillis / 1000.0
    println(s"\nAll animals finished! Total time with $threadCount thread(s): $duration seconds")
  } finally {
    executorService.shutdown()
  }
}

// Run with different thread pool sizes
runWithThreads(1)  // Sequential execution
runWithThreads(3)  // Parallel execution


=== Running with 1 thread(s) ===

🦁 Lion starting work...
🦁 Lion finished!
🦊 Fox starting work...
🦊 Fox finished!
🐇 Rabbit starting work...
🐇 Rabbit finished!

All animals finished! Total time with 1 thread(s): 3.012 seconds

=== Running with 3 thread(s) ===

🦁 Lion starting work...
🦊 Fox starting work...
🐇 Rabbit starting work...
🐇 Rabbit finished!
🦁 Lion finished!
🦊 Fox finished!

All animals finished! Total time with 3 thread(s): 1.009 seconds


defined function cpuBoundTask
defined function runWithThreads

## 3. Deadlock Demonstration

This example shows a classic deadlock scenario where two threads each hold a resource the other needs.

**Note:** We won't actually run this in the notebook as it would deadlock. Instead, let's look at the code and understand it.

**File: `thread_deadlock.scala`**

## 4. Error Handling

Futures can fail with an exception, resulting in a `Failure`. By default, unhandled exceptions in Futures are reported to the `ExecutionContext`'s `reportFailure` method (often just logging them), but they don't necessarily crash the program or affect other concurrent Futures.

Proper error handling involves using `recover` or `recoverWith` to handle exceptions or, more robustly, returning `Future[Either[Error, Success]]`.

In [23]:
import scala.concurrent._
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global
import java.time.{Duration, Instant}
import scala.util.{Try, Success, Failure} // Import Try

// Redefine the main logic using Futures for better async style
// We'll keep the original structure and refactor the 'mammal' part
// to avoid Await.result inside the Future.

println("🌍 World: 🚀 starting async hierarchy demo")
val worldStart = Instant.now()

// Mammal branch (using Futures throughout, handling bear failure asynchronously)
val mammal: Future[Unit] = Future { // Defines the mammal Future, starts its work
  val mammalStart = Instant.now()
  println("  🐾 Mammal: 🚀 started (child of World)")

  // Animal Futures - start immediately when created
  val lion: Future[Unit] = Future {
    val start = Instant.now()
    println("    🦁 Lion: 🚀 started (child of Mammal)")
    Thread.sleep(100)
    println(s"    🦁 Lion: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
  }

  val tiger: Future[Unit] = Future {
    val start = Instant.now()
    println("    🐯 Tiger: 🚀 started (child of Mammal)")
    Thread.sleep(100)
    println(s"    🐯 Tiger: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
  }

  // Bear Future with nested fruit Futures and an exception
  val bear: Future[Unit] = Future { // Bear Future starts immediately
    val bearStart = Instant.now()
    println("    🐻 Bear: 🚀 started (child of Mammal)")

    // Fruit Futures - These start immediately when created *within* the bear's computation.
    // Their lifecycle is independent of the bear Future's *Success/Failure state* once created.
    val apple: Future[Unit] = Future {
      val start = Instant.now()
      println("      🍎 Apple: 🚀 started (child of Bear logic)") // Clarify relationship
      Thread.sleep(550) // Long task - likely orphaned
      println(s"      🍎 Apple: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
    }

    val banana: Future[Unit] = Future {
      val start = Instant.now()
      println("      🍌 Banana: 🚀 started (child of Bear logic)") // Clarify relationship
      Thread.sleep(150) // Shorter task
      println(s"      🍌 Banana: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
    }

    val cherry: Future[Unit] = Future {
      val start = Instant.now()
      println("      🍒 Cherry: 🚀 started (child of Bear logic)") // Clarify relationship
      Thread.sleep(50) // Quick task
      println(s"      🍒 Cherry: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
    }

    Thread.sleep(100) // Simulate work before panic

    // Check status of fruit futures at the time the bear panics
    // Note: isCompleted only tells you if the Future has *finished* (success or failure).
    // It doesn't mean the work didn't start if not completed.
    println(s"    🐻 Bear: 💥 panicking now! Fruits finished: " +
            s"🍎${apple.isCompleted}, 🍌${banana.isCompleted}, 🍒${cherry.isCompleted}")

    throw new RuntimeException("Bear panicked: bear hates apples!") // Bear Future fails here
  }

  // --- Handle bear's outcome asynchronously using onComplete ---
  // This callback will run when the 'bear' Future completes (in this case, with Failure).
  // It does NOT block the thread running the mammal Future.
  bear.onComplete {
    case Success(_) =>
       // This block won't be reached in this specific scenario as bear always fails
       println("    🐻 Bear: ✅ finished normally")
    case Failure(e) =>
       println(s"    🐻 Bear: 💥 panicked and was handled asynchronously - ${e.getMessage}")
  }

  // --- Ensure the Mammal Future waits for its children Futures to *complete their lifecycle* ---
  // We want the mammal Future to only finish after lion, tiger, and bear's Futures are done
  // (either success or failure). We use transform to convert Bear's Failure into a Success(Unit)
  // so that Future.sequence doesn't immediately fail the mammal Future just because bear failed.
  val bearOutcomeAsSuccess: Future[Unit] = bear.transform {
    case Success(_) => Success(()) // If bear succeeded, return Success(Unit)
    case Failure(_) => Success(()) // If bear failed, return Success(Unit) - the failure message was logged by onComplete
  }

  // Combine the completion signals of all immediate children (Lion, Tiger, and the Bear's outcome as a Success)
  // Future.sequence will wait for all of these futures to complete (either original Success or transformed Success)
  val allMammalChildrenCompleted: Future[Seq[Unit]] = Future.sequence(Seq(lion, tiger, bearOutcomeAsSuccess))

  // Map the final result (Seq[Unit]) to Unit and print the Mammal completion message.
  // This map's Future[Unit] result is the final result of the 'mammal' Future.
  allMammalChildrenCompleted.map { _ =>
    println(s"  🐾 Mammal: ✅ finished (all children done) in ${Duration.between(mammalStart, Instant.now()).toMillis} ms")
  }
  // The return value of this block is the result of the map, which is a Future[Unit].
  // This Future completes when allMammalChildrenCompleted completes.

} // End of mammal Future block


// Bird branch (keeping the original structure for contrast, still uses Await.result internally)
val bird: Future[Unit] = Future {
  val birdStart = Instant.now()
  println("  🐦 Bird: 🚀 started (child of World)")

  // Eagle Future
  val eagle = Future { // Future starts immediately
    val start = Instant.now()
    println("    🦅 Eagle: 🚀 started (child of Bird)")
    Thread.sleep(100)
    println(s"    🦅 Eagle: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
  }

  // Sparrow Future
  val sparrow = Future { // Future starts immediately
    val sparrowStart = Instant.now()
    println("    🐦 Sparrow: 🚀 started (child of Bird)")

    // Worm as a sub-task (child of Sparrow logic)
    val worm = Future { // Future starts immediately
      val start = Instant.now()
      println("      🪱 Worm: 🚀 started (child of Sparrow logic)")
      Thread.sleep(300)
      println(s"      🪱 Worm: ✅ finished in ${Duration.between(start, Instant.now()).toMillis} ms")
    }

    // Sparrow *blocks* waiting for its child worm Future to complete
    Await.result(worm, 1.second) // This is a blocking call inside the sparrow Future
    println(s"    🐦 Sparrow: ✅ finished in ${Duration.between(sparrowStart, Instant.now()).toMillis} ms")
  }

  // Bird *blocks* waiting for its child Futures eagle and sparrow to complete
  Await.result(eagle, 1.second) // This is blocking inside the bird Future
  Await.result(sparrow, 1.second) // This is blocking inside the bird Future
  println(s"  🐦 Bird: ✅ finished (all children done) in ${Duration.between(birdStart, Instant.now()).toMillis} ms")
} // End of bird Future block


// World awaits both main branches using Await.result (acceptable in main/top-level)
// We await the top-level Futures which themselves manage their children asynchronously (mammal)
// or by blocking internally (bird).
Await.result(mammal, Duration.ofSeconds(5).toMillis.millis)
Await.result(bird, Duration.ofSeconds(5).toMillis.millis)

// Final note about the orphaned apple task
println(s"🌍 World: ✅ finished in ${Duration.between(worldStart, Instant.now()).toMillis} ms")
println("Note: The Apple task may continue running after its parent (Bear) Future completed with failure!")
// To verify if Apple finished, you would ideally need to keep a reference to the 'apple' Future
// outside the bear block and check its status after the main Await.result calls.
// For this example, the principle of orphaned tasks is demonstrated by the apple's long sleep
// and the fact that the bear finishes (with error) much earlier.
Thread.sleep(500)
println("\n--- End of Async Hierarchy Demo ---")

🌍 World: 🚀 starting async hierarchy demo
  🐾 Mammal: 🚀 started (child of World)
  🐦 Bird: 🚀 started (child of World)
    🦁 Lion: 🚀 started (child of Mammal)
    🦅 Eagle: 🚀 started (child of Bird)
    🐦 Sparrow: 🚀 started (child of Bird)
    🐯 Tiger: 🚀 started (child of Mammal)
    🐻 Bear: 🚀 started (child of Mammal)
      🪱 Worm: 🚀 started (child of Sparrow logic)
      🍎 Apple: 🚀 started (child of Bear logic)
      🍌 Banana: 🚀 started (child of Bear logic)
      🍒 Cherry: 🚀 started (child of Bear logic)
      🍒 Cherry: ✅ finished in 55 ms
    🦁 Lion: ✅ finished in 100 ms
    🐯 Tiger: ✅ finished in 101 ms
    🐻 Bear: 💥 panicking now! Fruits finished: 🍎false, 🍌false, 🍒true
    🦅 Eagle: ✅ finished in 101 ms
    🐻 Bear: 💥 panicked and was handled asynchronously - Bear panicked: bear hates apples!
  🐾 Mammal: ✅ finished (all children done) in 101 ms
      🍌 Banana: ✅ finished in 155 ms
      🪱 Worm: ✅ finished in 305 ms
    🐦 Sparrow: ✅ finished in 306 ms
  🐦 Bird: ✅ finished (all children

import scala.concurrent._

import scala.concurrent.duration._

import scala.concurrent.ExecutionContext.Implicits.global

import java.time.{Duration, Instant}

import scala.util.{Try, Success, Failure} // Import Try

// Redefine the main logic using Futures for better async style
// We'll keep the original structure and refactor the 'mammal' part
// to avoid Await.result inside the Future.


worldStart: Instant = 2025-05-06T04:37:00.982849Z
mammal: Future[Unit] = Success(value = ())
bird: Future[Unit] = Success(value = ())

## 5. Best Practices: Future[Either] Pattern

This example demonstrates recommended patterns for working with Scala's Futures, focusing on proper error handling with the `Future[Either[Error, Success]]` pattern.

**File: `future.scala`**

In [7]:
// Domain classes for our example
case class Animal(name: String, species: String)
case class Diet(foodType: String, dailyAmount: Int)

// Error model with sealed trait
sealed trait ZooError {
  def message: String
}
case class DatabaseError(message: String) extends ZooError
case class ValidationError(message: String) extends ZooError

// Demo of the Future[Either] pattern
def demoFutureEither(): Unit = {
  println("\n=== Future[Either] Best Practice Demo ===\n")
  
  // Step 1: Service returning Future[Either]
  def findAnimal(name: String): Future[Either[ZooError, Animal]] = Future {
    println(s"🔍 Searching for animal: $name")
    
    name match {
      case "lion" => Right(Animal("Leo", "Lion"))
      case "tiger" => Right(Animal("Tigra", "Tiger"))
      case _ => Left(ValidationError(s"Unknown animal: $name"))
    }
  }
  
  // Step 2: Another service returning Future[Either]
  def getDiet(animal: Animal): Future[Either[ZooError, Diet]] = Future {
    println(s"🥩 Fetching diet for: ${animal.name} the ${animal.species}")
    
    animal.species match {
      case "Lion" => Right(Diet("Meat", 10))
      case "Tiger" => Right(Diet("Meat", 8))
      case _ => Left(ValidationError(s"No diet info for ${animal.species}"))
    }
  }
  
  // Step 3: Combining futures with for-comprehension
  def getAnimalWithDiet(name: String): Future[Either[ZooError, (Animal, Diet)]] = {
    for {
      // Find animal
      animalResult <- findAnimal(name)
      
      // If animal found, get diet (otherwise short-circuit)
      dietResult <- animalResult match {
        case Right(animal) => getDiet(animal).map(diet => diet.map(d => (animal, d)))
        case Left(error) => Future.successful(Left(error))
      }
    } yield dietResult
  }
  
  // Test with successful case
  val lionResult = getAnimalWithDiet("lion")
  Await.result(lionResult, 2.seconds) match {
    case Right((animal, diet)) => 
      println(s"✅ Success: ${animal.name} the ${animal.species} eats ${diet.foodType}")
    case Left(error) => 
      println(s"❌ Error: ${error.message}")
  }
  
  // Test with error case
  val unicornResult = getAnimalWithDiet("unicorn")
  Await.result(unicornResult, 2.seconds) match {
    case Right((animal, diet)) => 
      println(s"✅ Success: ${animal.name} the ${animal.species} eats ${diet.foodType}")
    case Left(error) => 
      println(s"❌ Error: ${error.message}")
  }
}

demoFutureEither()

      🍌 Banana: Finished
      🍎 Apple: Finished

=== Future[Either] Best Practice Demo ===

🔍 Searching for animal: lion
🥩 Fetching diet for: Leo the Lion
✅ Success: Leo the Lion eats Meat
🔍 Searching for animal: unicorn
❌ Error: Unknown animal: unicorn


defined class Animal
defined class Diet
defined trait ZooError
defined class DatabaseError
defined class ValidationError
defined function demoFutureEither

## 6. Running External Scala Files

We can also run our complete Scala examples using shell commands from the notebook:

In [24]:
// Run the await.scala example (if you want to run it externally)
// Uncomment to execute
import sys.process._

"scala await.scala". !!

import sys.process._


res24_1: String = """🦁 lionRuns() is called.
🦊 foxRuns() is called.
🦁 Lion is running!
🐇 rabbitRuns() is called.
🦊 Fox is running!
(All animals are running already in Scala Future!)
🐇 Rabbit is running!
All animals have finished running (even the rabbit 🐇, because Scala Future is eager)!

--- Lazy version ---
🦁 lazyLionRuns() is called.
🦊 lazyFoxRuns() is called.
🐇 lazyRabbitRuns() is called.
(No animals are running yet...)
🦊 Fox is running!
🦁 Lion is running!
All animals have finished running (except the rabbit 🐇, who never started)!
"""

## Key Differences: Scala vs Rust Async

| Aspect | Scala Future | Rust Async/Await |
|--------|-------------|------------------|
| Execution Model | **Eager**: Starts executing as soon as created | **Lazy**: Only starts when awaited |
| Type System | Uses `Future[T]` for all async results | Uses `impl Future<Output=T>` and other future types |
| Error Handling | Usually with `Try`, `Either`, or exceptions | With `Result<T, E>` return types |
| Cancellation | No built-in cancellation | Dropping a future can cancel it |
| Composition | `map`, `flatMap`, for-comprehensions | Async blocks, `.await` syntax |
| Runtime | Requires an `ExecutionContext` | Requires a runtime like Tokio |
| Resource Efficiency | Each future is a lightweight task | Zero-cost abstraction with state machines |

## Learning Resources

* [Introduction to scala-async](https://www.baeldung.com/scala/scala-async)

* [Working with Futures in Scala: A Quick Introduction](https://towardsdev.com/working-with-futures-in-scala-a-quick-introduction-9223703ab25e)

* [Synchronous Handling of Futures](https://www.baeldung.com/scala/synchronous-handling-of-futures)

### Sync, Async, Blocking and Non-Blocking | Rock the JVM

[![Sync, Async, Blocking and Non-Blocking | Rock the JVM](https://img.youtube.com/vi/Hlu-zYeNsSU/0.jpg)](https://www.youtube.com/watch?v=Hlu-zYeNsSU)

[Watch on YouTube](https://www.youtube.com/watch?v=Hlu-zYeNsSU)

---

### Adam Rosien - Async/Await for the Monadic Programmer | Scala Days 2023 Seattle

[![Adam Rosien - Async/Await for the Monadic Programmer | Scala Days 2023 Seattle](https://img.youtube.com/vi/OH5cxLNTTPo/0.jpg)](https://www.youtube.com/watch?v=OH5cxLNTTPo)

[Watch on YouTube](https://www.youtube.com/watch?v=OH5cxLNTTPo)

---

### Scala Programming - Introduction to Threads and Futures

[![Scala Programming - Introduction to Threads and Futures](https://img.youtube.com/vi/6b24sszy6Js/0.jpg)](https://youtu.be/6b24sszy6Js)

[Watch on YouTube](https://youtu.be/6b24sszy6Js)